In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms

In [ ]:
class BasicConv2d(nn.Module):
  def __init__(self, in_channel:int, out_channel:int,*args, **kwargs):
    super().__init__()

    self.cn = nn.Conv2d(in_channel, out_channel,*args, **kwargs)
    self.bn = nn.BatchNorm2d(out_channel) # barcgNormalization befor activation.number of features= output_channel
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.cn(x)
    x = self.bn(x)
    x = self.relu(x)
    return x

x = torch.randn(32, 3 , 224,224)
model = BasicConv2d(3 ,64 ,3 ,1 ,0) # input_channel, output_channel,kernnel,  stride, padding
model(x).size()

torch.Size([32, 64, 222, 222])

batch size=32 , output_channel=64, window_size = 222*222

---



In [ ]:
class Inception(nn.Module):
    def __init__(self, in_channel: int,
                 b1_11_ch: int, # first branch: kernel 1*1
                 b2_11_ch: int, b2_33_ch: int, # int: output_channel
                 b3_11_ch: int, b3_55_ch: int,
                 b4_11_ch: int):
      super().__init__()
      self.b1 = BasicConv2d(in_channel, b1_11_ch, 1, 1, 0)#b1_11_ch: output_channel, kernel, stride,same padding
      self.b2 = nn.Sequential(
          BasicConv2d(in_channel, b2_11_ch, 1, 1, 0),
          BasicConv2d(b2_11_ch, b2_33_ch, 3, 1, 1),
      )
      self.b3 = nn.Sequential(
          BasicConv2d(in_channel, b3_11_ch, 1, 1, 0),
          BasicConv2d(b3_11_ch, b3_55_ch, 5, 1, 2),
      )
      self.b4 = nn.Sequential(
          nn.MaxPool2d(kernel_size=(3, 3), stride=1, padding=1),
          BasicConv2d(in_channel, b4_11_ch, 5, 1, 2),
      )

    def forward(self,x):
        b1 = self.b1(x)
        b2 = self.b2(x)
        b3 = self.b3(x)
        b4 = self.b4(x)
        x = torch.cat([b1, b2, b3, b4], 1) #0: barch_size, 1:channel, 2:height , 3:width
        return x

x = torch.rand(32, 64, 32 ,32)
model = Inception(64 ,64, 96, 128, 12, 32, 32 )
# out_ch = 64  128+ 32 + 32 = 256
model(x).size()

torch.Size([32, 256, 32, 32])

concat of outputs branches.

In [ ]:
class MinorGoogleNet(nn.Module):
  def __init__(self, in_channel, num_classes: int):
    super().__init__()
    self.cn1 = BasicConv2d(in_channel, 64, 7, 2, 3)
    self.s1 = nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1)
    # size=5, alpha=0.0001, beta=0.75, k = 2
    self.lrn = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k = 2)

    self.inception1 = Inception(64, 64, 96, 128, 12, 32, 32)
    self.inception2 = Inception(256, 384, 192, 384, 48, 128, 128)
    # 384 + 384 + 128 + 128 = 1024

    self.gavgpool = nn.AdaptiveAvgPool2d((1,1)) # window size become 1*1. here we have 1024 outputs
    self.flatten = nn.Flatten() # batch size remaind the same as before
    self.dropout = nn.Dropout(0.4)
    self.classifier = nn.Linear(1024, num_classes) # 1024= input, num_class = output


  def forward(self, x:torch.Tensor) -> torch.Tensor:
    x = self.cn1(x)
    x = self.s1(x)
    x = self.lrn(x)
    x = self.inception1(x)
    x = self.inception2(x)
    x = self.gavgpool(x)
    x = self.flatten(x)
    x = self.dropout(x)
    x = self.classifier(x)
    return x

x = torch.randn(32, 3, 224, 224)
model = MinorGoogleNet(3, 1000)
model(x).size()

torch.Size([32, 1000])

batch_size=32, output=1000 beacause it has 1000 classes to predict

In [ ]:
models.